In [1]:
!pip install wandb

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np # linear algebra
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import torch.nn.functional as F
import wandb

Застосування:
Медичні зображення: Bicubic добре підходить для задач, де важливі дрібні деталі, наприклад, аналіз родимок, рентгенівських знімків, МРТ тощо.
Графіка та фотографія: Використовується для масштабування фотографій, щоб уникнути пікселізації.

In [2]:
class CNN_Autoencoder(nn.Module):
    def __init__(self):
        super(CNN_Autoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),  # Output: (batch_size, 32, 300, 225)
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # Output: (batch_size, 64, 150, 113)
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # Output: (batch_size, 128, 75, 56)
            nn.ReLU(),
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            # The first transpose convolution will output (batch_size, 64, 150, 113)
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  
            nn.ReLU(),
            # The second transpose convolution will output (batch_size, 32, 300, 226)
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=0),  
            nn.ReLU(),
            # The final transpose convolution will output (batch_size, 3, 600, 450)
            nn.ConvTranspose2d(32, 3, kernel_size=3, stride=2, padding=1, output_padding=0),  
            nn.Sigmoid()  # For normalization within [0, 1]
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [3]:
# Функція для обчислення коефіцієнта кореляції
def correlation_coefficient(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sqrt(np.sum((x - x_mean)**2) * np.sum((y - y_mean)**2))
    
    return numerator / denominator

# Функція для обчислення порогу
def calculate_threshold(correlation_coeffs):
    mu_c = np.mean(correlation_coeffs)
    sigma_c = np.std(correlation_coeffs)
    threshold = mu_c - 0.5 * sigma_c
    return threshold

In [4]:
!wandb login 492953ddcda0576b6e6ebf89860aed0ccd177efe

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
run_name = f"Autoencoder 1000 examples"

In [6]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Skin cancer BASE model",

    # track hyperparameters and run metadata
    config={
    "epochs" : 40,
    "batch_size" : 16,
    "learning_rate" : 0.001,
    },
    name=run_name
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kkaterynabilyk (kkaterynabilyk-lviv-polytechnic-national-university). Use `wandb login --relogin` to force relogin


In [7]:
# Клас для завантаження зображень з папки
class CustomDataset(Dataset):
    def __init__(self, folder_path, transform=None, max_images=None):
        self.image_paths = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path) if fname.endswith('.jpg') or fname.endswith('.png')]
        if max_images is not None:
            self.image_paths = self.image_paths[:max_images]  # Обмежуємо кількість зображень до max_images
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = self.image_paths[idx]
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [8]:
# Підготовка даних
image_folder = "/kaggle/input/ham1000-segmentation-and-classification/images"  # Шлях до папки з зображеннями
transform = transforms.Compose([
    transforms.Resize((600, 450), interpolation=transforms.InterpolationMode.BICUBIC),
    # transforms.CenterCrop(224),  # Не потрібно, якщо не хочете обрізати зображення
    transforms.ToTensor(),
    # Якщо не потрібно нормалізувати, можете прибрати:
    # transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)) 
])

dataset = CustomDataset(image_folder, transform)
dataloader = DataLoader(dataset, batch_size=wandb.config.batch_size, shuffle=True)

# # Створення та тренування автоенкодера
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder = CNN_Autoencoder().to(device)
optimizer = optim.Adam(autoencoder.parameters(), lr=wandb.config.learning_rate)
criterion = nn.MSELoss()  # Використовуємо BCE для бінарної класифікації
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=4, factor=0.1)

for epoch in range(wandb.config.epochs): 
    autoencoder.train()
    train_loss = 0.0
    
    for data in dataloader:
        data = data.to(device)
        
        optimizer.zero_grad()
        output = autoencoder(data)
        
        # Враховуємо розмір для уникнення помилки розміру
        data_resized = F.interpolate(data, size=(597, 453), mode='bicubic', align_corners=False)
        
        loss = criterion(output, data_resized)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Виведення тренувальних втрат
    print(f"Epoch [{epoch+1}/{wandb.config.epochs}], Loss: {train_loss/len(dataloader)}")

    # Оновлення швидкості навчання
    scheduler.step(train_loss/len(dataloader))

# Збереження моделі
torch.save(autoencoder.state_dict(), "cnn_autoencoder.pth")

Epoch [1/40], Loss: 0.0036228455609817523
Epoch [2/40], Loss: 0.0007202210491143774
Epoch [3/40], Loss: 0.00042115589325296166
Epoch [4/40], Loss: 0.00036933383727356933


KeyboardInterrupt: 

In [58]:
# Отримання вихідного зображення після реконструкції
test_image = Image.open("/kaggle/input/dddddsd/photo_2025-01-05_14-25-25.jpg").convert('RGB')
test_image = transform(test_image).unsqueeze(0).to(device)
output_image = autoencoder(test_image).detach().cpu().numpy()

# Обчислення коефіцієнту кореляції
test_image_resized = transforms.functional.resize(test_image, (597, 453), interpolation=transforms.InterpolationMode.BICUBIC)
input_pixels = test_image_resized.squeeze(0).cpu().numpy().flatten()

reconstructed_pixels = output_image.flatten()
corr_value = correlation_coefficient(input_pixels, reconstructed_pixels)
print(f"Correlation Coefficient: {corr_value}")

# Масив кореляцій для класу (приклад)
correlation_coeffs = np.random.rand(100)  # Масив кореляцій для класу

# Обчислення порогу для цього класу
threshold = calculate_threshold(correlation_coeffs)
print(f"Threshold: {threshold}")

# Перевірка чи зображення належить тому ж класу
if corr_value > threshold:
    print("Зображення належить тому ж класу.")
else:
    print("Зображення не належить тому ж класу.")

Correlation Coefficient: 0.8697779178619385
Threshold: 0.35221836510738785
Зображення належить тому ж класу.
